In [140]:
import numpy as np
import os

In [141]:
def load_data(folder_path):
    x_train = np.load(os.path.join(folder_path, 'x_train.npy'))
    y_train = np.load(os.path.join(folder_path, 'y_train.npy'))
    x_test = np.load(os.path.join(folder_path, 'x_test.npy'))
    y_test = np.load(os.path.join(folder_path, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [142]:
x_train, y_train, x_test, y_test = load_data('lr4_dataset/')

В данной лабораторной работе будет практиковаться поиск гиперпараметров. Буду рассмотрены алгоритмы поиска гиперпараметров: grid search, random search.

Помимо поиска гиперпараметров будет рассмотрен алгоритм кросс-валидации, позволяющий получить более достоверную оценку качества модели в условиях недостатка данных.
Хотя в работе предоставлена тестовая выборка, здесь она имеет сугубо теоретический характер (для получения финальной оценки) и на практике как правило недоступна. Поэтому во время подбора гиперпараметров используются лишь `x_train, y_train`. `x_test, y_test` используются лишь для получения финальной оценки, чтобы можно было видеть разницу между разными алгоритмами подбора гиперпараметров (если она будет).

Выберите одну модель из списка: MLPClassifier, SGDClassifier, DecisionTreeClassifier, RandomForestClassifier, SVC.
Для выбранной модели произведите поиск оптимальных гиперпараметров. 

**Требование**: поиск должен идти как минимум для двух гиперпараметров.

**Требование**: в конструктор моделей передавайте `random_state=1` для воспроизводимости результатов.

## 0. Обучение бейзлайн модели для проведения сравнения

In [143]:
# Обучите базовую модель без изменения гиперпараметров (т.е. используются гиперпараметры по умолчанию).
# Проанализируйте качество модели (accuracy, матрица ошибок).

# Напишите ваш код здесь.
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

baseline_model = SVC(random_state=1)
baseline_model.fit(x_train, y_train)

y_pred_baseline = baseline_model.predict(x_test)

accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
conf_matrix_baseline = confusion_matrix(y_test, y_pred_baseline)

print(f"Baseline Model Accuracy: {accuracy_baseline}")
print(f"Baseline Model Confusion Matrix:\n{conf_matrix_baseline}")

Baseline Model Accuracy: 0.7333333333333333
Baseline Model Confusion Matrix:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 1 0 0 1 0 0 1 0]
 [0 0 1 0 0 0 0 1 0 1]]


## 1. K-Fold Cross-Validation

In [144]:
# Реализуйте фунцию кросс-валидации

# Замечание: x_test, y_test не должны применятся в рамках данной функции.

def kfold_cv(model_fn, eval_fn, x: np.ndarray, y: np.ndarray, n_splits=5) -> float:
    """
    Parameters
    ----------
    model_fn : callable
        Функция-фабрика, что конструирует и возвращает новый объект модели.
        Например: `lambda: MLPClassifier(hidden_layer_sizes=(256,))`.
    eval_fn : callable
        Функция вида `eval_fn(labels, predictions)`, что возвращает скаляр (значение метрики).
    x : np.ndarray
        Набор признаков (размерность NxD, N - количество экземпляров, D - количество признаков).
    y : np.ndarray
        Набор меток (размерность N)
    n_splits : int, optional
        Количество фолдов (подвыборок), по умолчанию 5.

    Returns
    -------
    float
        Среднее значение метрики (что вычисляется eval_fn) по фолдам.
    """
    fold_size = len(x) // n_splits
    scores = []

    for fold in range(n_splits):
        val_start = fold * fold_size
        val_end = (fold + 1) * fold_size

        x_val = x[val_start:val_end]
        y_val = y[val_start:val_end]
        x_train = np.concatenate([x[:val_start], x[val_end:]])
        y_train = np.concatenate([y[:val_start], y[val_end:]])

        model = model_fn()
        model.fit(x_train, y_train)

        y_pred = model.predict(x_val)
        score = eval_fn(y_val, y_pred)
        scores.append(score)

    return np.mean(scores)

In [145]:
# Убедитесь в корректности работы функции кросс-валидации.

# Напишите ваш код здесь.
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

model_fn = lambda: SVC(random_state=1)
eval_fn = accuracy_score

cv_score = kfold_cv(model_fn, eval_fn, x_train, y_train)
print(f"Cross-Validation Score: {cv_score}")

Cross-Validation Score: 0.4454545454545455


## 2. Grid search

In [146]:
# 1. Реализуйте алгоритм поиска гиперпараметров grid search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации поиска гиперпараметров.

# Напишите ваш код здесь.
import itertools
import time

def grid_search(model_fn, param_grid, x_train, y_train, eval_fn, n_splits=5):
    best_score = -np.inf
    best_params = None
    all_scores = []

    for params in itertools.product(*param_grid.values()):
        current_params = dict(zip(param_grid.keys(), params))
        model = model_fn(**current_params)

        score = kfold_cv(lambda: model, eval_fn, x_train, y_train, n_splits)
        all_scores.append((current_params, score))

        if score > best_score:
            best_score = score
            best_params = current_params

    return best_params, all_scores

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

start_time = time.time()
best_params, all_scores = grid_search(
    model_fn=lambda **kwargs: SVC(random_state=1, **kwargs),
    param_grid=param_grid,
    x_train=x_train,
    y_train=y_train,
    eval_fn=accuracy_score,
    n_splits=5
)
end_time = time.time()

print(f"Best Parameters: {best_params}")
print(f"Grid Search Time: {end_time - start_time} seconds")

Best Parameters: {'C': 0.1, 'kernel': 'linear', 'gamma': 'scale'}
Grid Search Time: 0.19963932037353516 seconds


In [147]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.

# Напишите ваш код здесь.
best_model = SVC(**best_params, random_state=1)
best_model.fit(x_train, y_train)

y_pred_grid = best_model.predict(x_test)

accuracy_grid = accuracy_score(y_test, y_pred_grid)
conf_matrix_grid = confusion_matrix(y_test, y_pred_grid)

print(f"Grid Search Model Accuracy: {accuracy_grid}\n")
print(f"Grid Search Model Confusion Matrix:\n{conf_matrix_grid}\n")

print(f"Baseline Model Accuracy: {accuracy_baseline}")

#GridSearch дает значительно большую точность (0.76), чем модель с параметрами по умолчанию (0.73)

Grid Search Model Accuracy: 0.7666666666666667

Grid Search Model Confusion Matrix:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 0 2 0 0 1 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 1 0 1 0 1]]

Baseline Model Accuracy: 0.7333333333333333


## 3. Random search

In [148]:
# 1. Реализуйте алгоритм поиска гиперпараметров random search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации поиска гиперпараметров.
# Требование: количество итераций должно быть меньше в сравнении с grid search.

# Напишите ваш код здесь.
import random

def random_search(model_fn, param_dist, x_train, y_train, eval_fn, n_iter=10, n_splits=5):
    best_score = -np.inf
    best_params = None
    all_scores = []

    for _ in range(n_iter):
        current_params = {param: random.choice(values) for param, values in param_dist.items()}
        model = model_fn(**current_params)

        score = kfold_cv(lambda: model, eval_fn, x_train, y_train, n_splits)
        all_scores.append((current_params, score))

        if score > best_score:
            best_score = score
            best_params = current_params

    return best_params, all_scores

param_dist = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

start_time = time.time()
best_params, all_scores = random_search(
    model_fn=lambda **kwargs: SVC(random_state=1, **kwargs),
    param_dist=param_dist,
    x_train=x_train,
    y_train=y_train,
    eval_fn=accuracy_score,
    n_iter=10,
    n_splits=5
)
end_time = time.time()

print(f"Best Parameters: {best_params}")
print(f"Random Search Time: {end_time - start_time} seconds")

Best Parameters: {'C': 1, 'kernel': 'linear', 'gamma': 'scale'}
Random Search Time: 0.1957845687866211 seconds


In [149]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

# Напишите ваш код здесь.
best_model_random = SVC(**best_params_random, random_state=1)
best_model_random.fit(x_train, y_train)

y_pred_random = best_model_random.predict(x_test)

accuracy_random = accuracy_score(y_test, y_pred_random)
conf_matrix_random = confusion_matrix(y_test, y_pred_random)

print(f"Random Search Model Accuracy: {accuracy_random}\n")
print(f"Random Search Model Confusion Matrix:\n{conf_matrix_random}\n")

print(f"Baseline Model Accuracy: {accuracy_baseline}")
print(f"Grid Search Model Accuracy: {accuracy_grid}\n")


#RandomSearch дает большую точность (0.767), чем модель с параметрами по умолчанию (0.73)

Random Search Model Accuracy: 0.7666666666666667

Random Search Model Confusion Matrix:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 0 2 0 0 1 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 1 0 1 0 1]]

Baseline Model Accuracy: 0.7333333333333333
Grid Search Model Accuracy: 0.7666666666666667



## 4. Доп. задание (опционально)

### 4.1 Bayesian optimization

Примените байесовскую оптимизацию для поиска гиперпараметров.
В качестве алгоритма используйте `BayesSearchCV` из пакета `scikit-optimize`.

Сложность: почти бесплатный балл.

In [150]:
# 1. Инстанцируйте BayesSearchCV.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.

# Напишите ваш код здесь.
from skopt import BayesSearchCV
from skopt.space import Real, Categorical

bayes_param_grid = {
    'C': Real(0.1, 10),
    'kernel': Categorical(['linear', 'rbf']),
    'gamma': Categorical(['scale', 'auto'])
}

bayes_search = BayesSearchCV(
    estimator=SVC(random_state=1),
    search_spaces=bayes_param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    n_iter=10,
    random_state=1
)

start_time = time.time()
bayes_search.fit(x_train, y_train)
end_time = time.time()

bayes_search_time = end_time - start_time

best_params_bayes = bayes_search.best_params_
print(f"Best Parameters (Bayesian Optimization): {best_params_bayes}")
print(f"Bayesian Optimization Time: {bayes_search_time} seconds")

Best Parameters (Bayesian Optimization): OrderedDict({'C': 8.320950220852149, 'gamma': 'scale', 'kernel': 'rbf'})
Bayesian Optimization Time: 9.142077445983887 seconds


In [151]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

# Напишите ваш код здесь.
best_model_bayes = SVC(**best_params_bayes, random_state=1)
best_model_bayes.fit(x_train, y_train)

y_pred_bayes = best_model_bayes.predict(x_test)

accuracy_bayes = accuracy_score(y_test, y_pred_bayes)
conf_matrix_bayes = confusion_matrix(y_test, y_pred_bayes)

print(f"Bayesian Optimization Model Accuracy: {accuracy_bayes}\n")
print(f"Bayesian Optimization Model Confusion Matrix:\n{conf_matrix_bayes}\n")

print(f"Baseline Model Accuracy: {accuracy_baseline}")
print(f"Grid Search Model Accuracy: {accuracy_grid}\n")


#BayesSearchCV дает большую точность (0.8), чем модель с параметрами по умолчанию (0.73)

Bayesian Optimization Model Accuracy: 0.8

Bayesian Optimization Model Confusion Matrix:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 0 1]]

Baseline Model Accuracy: 0.7333333333333333
Grid Search Model Accuracy: 0.7666666666666667



### 4.2 Tree of Parzen Estimators (TPE) из HyperOpt

Примените TPE из библиотеки hyperopt для поиска гиперпараметров. Вики по HyperOpt: https://github.com/hyperopt/hyperopt/wiki/FMin

Сложность: чтец документаций o(*￣▽￣*)ブ.

In [152]:
def objective(args):
    # Принимает гиперпараметры, инстанцирует модель, обучает её, возвращает значение метрики.
    # Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
    
    # Напишите ваш код здесь.
    model = SVC(
        C=args['C'],
        kernel=args['kernel'],
        gamma=args['gamma'],
        random_state=1
    )
    model.fit(x_train, y_train)
    y_pred = model.predict(x_train)
    return -accuracy_score(y_train, y_pred)

In [153]:
# Определите пространство поиска гиперпараметров
from hyperopt import fmin, tpe, hp, Trials

space = {
    'C': hp.uniform('C', 0.1, 10),
    'kernel': hp.choice('kernel', ['linear', 'rbf']),
    'gamma': hp.choice('gamma', ['scale', 'auto'])
}

In [154]:
# 1. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 2. Выведите найденные значения гиперпараметров и время работы.

# Напишите ваш код здесь.
trials = Trials()
start_time = time.time()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)
end_time = time.time()

tpe_search_time = end_time - start_time

best_params_tpe = {
    'C': best['C'],
    'kernel': ['linear', 'rbf'][best['kernel']],
    'gamma': ['scale', 'auto'][best['gamma']]
}
print(f"Best Parameters (TPE): {best_params_tpe}")
print(f"TPE Search Time: {tpe_search_time} seconds")

100%|██████████| 10/10 [00:00<00:00, 101.02trial/s, best loss: -1.0]
Best Parameters (TPE): {'C': np.float64(8.934504270945704), 'kernel': 'linear', 'gamma': 'scale'}
TPE Search Time: 0.10500764846801758 seconds


In [155]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

# Напишите ваш код здесь.
best_model_tpe = SVC(**best_params_tpe, random_state=1)
best_model_tpe.fit(x_train, y_train)

y_pred_tpe = best_model_tpe.predict(x_test)

accuracy_tpe = accuracy_score(y_test, y_pred_tpe)
conf_matrix_tpe = confusion_matrix(y_test, y_pred_tpe)

print(f"TPE Model Accuracy: {accuracy_tpe}\n")
print(f"TPE Model Confusion Matrix:\n{conf_matrix_tpe}\n")


print(f"Baseline Model Accuracy: {accuracy_baseline}")
print(f"Grid Search Model Accuracy: {accuracy_grid}\n")
#TPE дает большую точность (0.767), чем модель с параметрами по умолчанию (0.73)

TPE Model Accuracy: 0.7666666666666667

TPE Model Confusion Matrix:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 0 2 0 0 1 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 1 0 1 0 1]]

Baseline Model Accuracy: 0.7333333333333333
Grid Search Model Accuracy: 0.7666666666666667



In [156]:
#Можно сделать вывод, что в большинстве случаев гиперпараметры по умолчанию уступает 
#Наилушим методом поиска гиперпараметров оказался BayesSearchCV c accuracy 0.8 для данного датасета